In [ ]:
!pip install -U gdown 
!pip install torchmetrics
!gdown --id 1n-YeSOxjOOKyhaU1OE3Bqp_HmDm3pIqq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1n-YeSOxjOOKyhaU1OE3Bqp_HmDm3pIqq
To: /content/file.csv
100% 354k/354k [00:00<00:00, 143MB/s]


In [ ]:
import pandas as pd
df = pd.read_csv("/content/file.csv")

In [ ]:
df_data = df[["TimeProcess","People", "H", "Positive", "Negative", "NH3", 'label']]


In [ ]:
df_data = (df_data-df_data.min(0)) / (df_data.max(0) - df_data.min(0))

In [ ]:
df_data

In [ ]:
df_data_label = df["NH3"]

In [ ]:
df_data[:5]

In [ ]:
len_train = int(len(df_data["NH3"])*0.8)

In [ ]:
train_data = df_data.values[:len_train]
train_label = df_data["H"].values[:len_train][:,None]

valid_data = df_data.values[len_train:]
valid_label = df_data["H"].values[len_train:][:,None]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN
import numpy as np
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 5
n_features = 7
generator = TimeseriesGenerator(train_data, train_label, length=n_input, batch_size=16)
val_generator = TimeseriesGenerator(valid_data, valid_label, length=n_input, batch_size=16)
# define model
model = Sequential()
model.add(LSTM(32, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))

from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
model.compile(optimizer='adam', loss='mae')
model.fit(generator, validation_data=val_generator,epochs=20)


```
nh3--
rnn 0.0906 lstm 0.0875 -- rmse * (df_data.max(0) - df_data.min(0)).values[5]
rnn 0.0740 lstm 0.0707 -- mae * (df_data.max(0) - df_data.min(0)).values[5]

rh--
rnn 0.0997 lstm 0.0982 -- rmse * (df_data.max(0) - df_data.min(0)).values[2]
rnn 0.0783 lstm 0.0773 -- mae * (df_data.max(0) - df_data.min(0)).values[2]


```